In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('german_credit_data.csv',index_col=0)
df.head(10)

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,49,male,1,own,little,NaN,2096,12,education,good
3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,53,male,2,free,little,little,4870,24,car,bad
5,35,male,1,free,NaN,NaN,9055,36,education,good
6,53,male,2,own,quite rich,NaN,2835,24,furniture/equipment,good
7,35,male,3,rent,little,moderate,6948,36,car,good
8,61,male,1,own,rich,NaN,3059,12,radio/TV,good
9,28,male,3,own,little,moderate,5234,30,car,bad


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 10 columns):
Age                 1000 non-null int64
Sex                 1000 non-null object
Job                 1000 non-null int64
Housing             1000 non-null object
Saving accounts     817 non-null object
Checking account    606 non-null object
Credit amount       1000 non-null int64
Duration            1000 non-null int64
Purpose             1000 non-null object
Risk                1000 non-null object
dtypes: int64(4), object(6)
memory usage: 85.9+ KB


In [4]:
#Changing NaN to other value for processing

df=df.fillna("aa")

In [5]:
#Preprocessing-Label encoding

from sklearn.preprocessing import LabelEncoder

labenc=LabelEncoder()
df['Sex']=labenc.fit_transform(df['Sex'])
df['Housing']=labenc.fit_transform(df['Housing'])
df['Saving accounts']=df['Saving accounts'].astype(str)
df['Saving accounts']=labenc.fit_transform(df['Saving accounts'])
df['Checking account']=df['Checking account'].astype(str)
df['Checking account']=labenc.fit_transform(df['Checking account'])
df['Purpose']=labenc.fit_transform(df['Purpose'])
df['Risk']=labenc.fit_transform(df['Risk'])

In [6]:
#Preprocessing- Encoding variable

col=pd.get_dummies(df['Purpose'],prefix='Purpose')
col=col.drop(col.columns[0],axis=1)
df=pd.concat([df,col],axis=1)
df=df.drop('Purpose',axis=1)

In [7]:
#Preprocessing-Categorising numerical variables

df['binss']=pd.cut(df['Age'],bins=10,labels=list(range(1,11)))
df['Age']=np.where((df['binss']==1)|(df['binss']==7),1,np.where((df['binss']==4)|(df['binss']==9),2,
  np.where((df['binss']==6)|(df['binss']==10),3,4)))

df['binss']=pd.cut(df['Credit amount'],bins=10,labels=list(range(1,11)))
df['Credit amount']=np.where((df['binss']==3)|(df['binss']==5)|(df['binss']==6),
  1,np.where((df['binss']==7)|(df['binss']==8),
             2,np.where((df['binss']==1),
                        3,np.where((df['binss']==2),
                                   4,np.where((df['binss']==4),5,6)))))

df['binss']=pd.cut(df['Duration'],bins=10,labels=list(range(1,11)))
df['Duration']=np.where((df['binss']==3)|(df['binss']==4)|(df['binss']==6),1,
  np.where(df['binss']==1,2,np.where(df['binss']==2,3,np.where(df['binss']==7,4,5))))

df=df.drop('binss',axis=1)

In [8]:
#Seperating Output variable 

y=df['Risk']
df=df.drop(['Risk'],axis=1)

In [9]:
#Over-Sampling Minority Group

from imblearn.over_sampling import SMOTE

sm=SMOTE(random_state=0,k_neighbors=5,ratio='minority')
df,y=sm.fit_sample(df,y)

In [10]:
#Selecting Significant Variable

x=df[:,[0,2,3,4,5,6,7]]

In [11]:
#Spliting Data into Test and Train

from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [12]:
#Random Forest Classification

from sklearn.ensemble import RandomForestClassifier

cl=RandomForestClassifier(random_state=6,max_depth=8, max_features='log2', min_samples_split=5, n_estimators=30)
cl.fit(x_train,y_train)
y_pred=cl.predict(x_test)

In [13]:
#Testing accuracy

from sklearn.metrics import accuracy_score,confusion_matrix,f1_score,roc_auc_score
from sklearn.model_selection import cross_val_score

cv_acc=cross_val_score(estimator=cl,X=x_train,y=y_train,cv=10)
cm=confusion_matrix(y_test,y_pred)
cv_acc=cv_acc.mean()
acc=accuracy_score(y_test,y_pred)
f1=f1_score(y_test,y_pred,average='binary')
auc=roc_auc_score(y_test,y_pred)
sen=cm[0][0]/(cm[0][0]+cm[1][0])
spec=cm[1][1]/(cm[1][1]+cm[0][1])

In [14]:
# Printing Output

print('\nTraining Accuracy:',cv_acc)
print('\nTesting Accuracy:',acc)
print('\nSensitivity',sen)
print('\nSpecificity',spec)
print('\nF1 Score:',f1)
print('\nAUC Score:',auc)
print('\nConfusion Matrix:\n',cm)


Training Accuracy: 0.7964179650573456

Testing Accuracy: 0.8

Sensitivity 0.7985611510791367

Specificity 0.8014184397163121

F1 Score: 0.8014184397163121

AUC Score: 0.7999897953977245

Confusion Matrix:
 [[111  28]
 [ 28 113]]
